In [1]:
%pip install qdrant-client sentence-transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 25.3
[notice] To update, run: C:\Users\kke\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
from qdrant_client import QdrantClient

client = QdrantClient(path="../qdrant_data")
print("Qdrant Local Mode läuft")

Qdrant Local Mode läuft


In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
dim = model.get_sentence_embedding_dimension()
print("Embedding-Dimension:", dim)
texts = [
    "Paris ist die Hauptstadt von Frankreich.",
    "Berlin ist die Hauptstadt von Deutschland.",
    "Rom ist die Hauptstadt von Italien."
]

print("Texte geladen:", len(texts))


C:\Users\kke\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embedding-Dimension: 384
Texte geladen: 3


In [4]:
from qdrant_client.models import VectorParams, Distance

COLLECTION = "demo_texts"

# Neu anlegen (für sauberen Start)
if client.collection_exists(COLLECTION):
    client.delete_collection(COLLECTION)

client.create_collection(
    collection_name=COLLECTION,
    vectors_config=VectorParams(size=dim, distance=Distance.COSINE)
)

print("Collection erstellt:", COLLECTION)


Collection erstellt: demo_texts


In [5]:
from qdrant_client.models import PointStruct

vectors = model.encode(texts, normalize_embeddings=True)

points = [
    PointStruct(
        id=i,
        vector=vectors[i].tolist(),
        payload={"text": texts[i]}
    )
    for i in range(len(texts))
]

client.upsert(collection_name=COLLECTION, points=points)

print("Gespeichert:", len(points), "Texte")


Gespeichert: 3 Texte


In [6]:
query = "Was ist die Hauptstadt von Frankreich?"
qvec = model.encode([query], normalize_embeddings=True)[0].tolist()

res = client.query_points(
    collection_name=COLLECTION,
    query=qvec,
    limit=3,
    with_payload=True
)

# je nach Version: res.points enthält die Treffer
hits = res.points if hasattr(res, "points") else res

print("🔎 Treffer:")
for h in hits:
    payload = h.payload if hasattr(h, "payload") else h.get("payload", {})
    score = h.score if hasattr(h, "score") else h.get("score")
    print("-", payload.get("text"), "(score:", round(score, 3), ")")


🔎 Treffer:
- Paris ist die Hauptstadt von Frankreich. (score: 0.822 )
- Berlin ist die Hauptstadt von Deutschland. (score: 0.672 )
- Rom ist die Hauptstadt von Italien. (score: 0.62 )
